In [1]:
import pandas as pd
import xarray as xr
import numpy as np

from tqdm.notebook import tqdm

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
for yr in tqdm(range(1981, 2021)):
    
    stm = pd.read_csv('../../data/storms_lodise/cyclone_tracks_lodise/SO_EC_tracks_' + str(yr) + '.csv')
    
    if yr == 1981:
        
        storms = stm
        
    else:

        stm['Storm ID'] = stm['Storm ID'] + storms['Storm ID'].max()
        
        storms = pd.concat([storms, stm])

ds_cyclones = storms.to_xarray()

ds_cyclones = ds_cyclones.assign_coords(TIME = ds_cyclones['Time'].astype(np.datetime64))
ds_cyclones = ds_cyclones.swap_dims({'index': 'TIME'})

ds_cyclones = ds_cyclones.rename({'Longitude': 'LONGITUDE',
                                  'Latitude': 'LATITUDE',
                                  'Storm ID': 'STORM_ID',
                                  'Minimum Pressure (hPa)': 'PRESSURE_MIN'})

ds_cyclones = ds_cyclones.drop(labels={'index', 'Time'})

ds_cyclones.attrs['dataset'] = 'Southern Ocean Extratropical Cyclone Tracks'
ds_cyclones.attrs['data source'] = 'https://github.com/jlodise/JGR2022_ExtratropicalCycloneTracker'
ds_cyclones.attrs['method'] = 'Lodise et al. 2022 - Global Climatology of Extratropical Cyclones From a New Tracking Approach and Associated Wave Heights from Satellite Radar Altimeter'
ds_cyclones.attrs['original data'] = 'ERA5 (https://www.ecmwf.int/en/forecasts/datasets/reanalysis-datasets/era5)'

  0%|          | 0/40 [00:00<?, ?it/s]

/var/folders/v4/3lf8sz_n06jf5z94ng5n_mnh0000gp/T/ipykernel_73079/1448243911.py:17: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  ds_cyclones = ds_cyclones.assign_coords(TIME = ds_cyclones['Time'].astype(np.datetime64))


In [4]:
ds_cyclones.to_netcdf('../../data/storms_lodise/ec_centers_1981_2020.nc')